In [1]:
import pandas as pd
from copy import deepcopy

In [2]:
train = pd.read_csv("./open/train.csv")
test = pd.read_csv("./open/test.csv")

In [3]:
from copy import deepcopy

train_df = deepcopy(train)
test_df = deepcopy(test)

In [4]:
drop_trn_cols= ["사고유형 - 세부분류",\
            "법규위반",\
            "가해운전자 차종",\
            "가해운전자 성별",\
            "가해운전자 연령",\
            "가해운전자 상해정도",\
            "피해운전자 차종",\
            "피해운전자 성별",\
            "피해운전자 연령",\
            "피해운전자 상해정도",\
            "사망자수",\
            "중상자수",\
            "경상자수",\
            "부상자수"]
train_df.drop(columns= drop_trn_cols, axis=1, inplace=True)

In [5]:
train_df["연월일"]= train_df["사고일시"].str.split(" ").str[0]
train_df["연"]= train_df["연월일"].str.split("-").str[0]
train_df["월"]= train_df["연월일"].str.split("-").str[1]
train_df["일"]= train_df["연월일"].str.split("-").str[2]

train_df["시"]= train_df["사고일시"].str.split(" ").str[1]


train_df.drop("사고일시", axis=1, inplace=True)
train_df.drop("연월일", axis=1, inplace=True)

In [6]:
train_df["구"]= train_df["시군구"].str.split().str[1]
train_df["동"]= train_df["시군구"].str.split().str[2]
train_df.drop("시군구", axis=1, inplace=True)

In [7]:
train_df.drop("ID", axis=1, inplace=True)
train_df.drop("ECLO", axis=1, inplace=True)

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
encoder = OneHotEncoder(sparse_output=False)

In [10]:
train_df_enc = encoder.fit_transform(train_df)

In [11]:
train_df_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [12]:
train_df_enc = pd.DataFrame(encoder.transform(train_df), columns=encoder.get_feature_names_out())

In [22]:
train_df_prep = pd.concat([ train_df_enc, train["ECLO"]], axis=1)

In [23]:
train_df_prep

,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,기상상태_기타,기상상태_눈,기상상태_맑음,...,동_학정동,동_향촌동,동_현풍읍,동_호림동,동_호산동,동_화원읍,동_화전동,동_황금동,동_효목동,ECLO
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
39605,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
39606,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
39607,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [27]:
import torch
import numpy as np
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
X_trn = torch.tensor(train_df_prep.to_numpy(dtype=np.float32))
# train_ds = TensorDataset(torch.tensor(train_df_prep).to_numpy(dtype=np.float32))
train_dl = DataLoader(X_trn)

In [32]:
import torch.nn as nn
import torch.nn.functional as F

class ANN(nn.Module):
    def __init__(self):
        super.__init__()
        self.lin1 = nn.Linear(308 ,1024)
        self.lin2 = nn.Linear(1024 ,1)
        self.activation = F.relu
    def forward(self, x):
        x = self.lin1(x)
        x = self.activation(x)
        x = self.lin2(x)
        
        return x

In [34]:
model = ANN

In [35]:
model(train_dl)

TypeError: ANN.__init__() takes 1 positional argument but 2 were given